<a href="https://colab.research.google.com/github/CoderCatA5/GuessFlat/blob/main/GuessFlat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MADKAUSING GUESSFLAT
```
Ayush Singh          -PES2UG20C0S81
Ayushmaan Kaushik    -PES2UG20CS082
Bhavini Madhuranath  -PES2UG20CS088
```

### An indian music genre classifier


# some exectued code we dont require
```
#linking google drive
from google.colab import drive
drive.mount('/content/drive')


import os
import pandas as pd
import matplotlib
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import librosa.display

os.getcwd()


moozyc_path='/content/drive/MyDrive/moozyc'
genre_list=os.listdir(moozyc_path)
#genre_list


data=[]
for genre in genre_list:
  genre_path='/'+genre
  music_files=os.listdir(moozyc_path+genre_path)
  music_files=[i  for i in music_files if i!='.spotdl-cache']
  
  #add each song to pandas df
  for song in music_files:
    song_path=moozyc_path+genre_path+'/'+song
    data.append([song_path,song,genre])

#creating pandas dataframe
song_df=pd.DataFrame(data,columns=['file_path','song','genre'])
song_df


%rm -rf /content/spectrograms6sec

os.mkdir('/content/spectrograms6sec')
for genre in genre_list:
  os.mkdir(f'/content/spectrograms6sec/{genre}')

#making spectrograms save in file
def save_spectrogram(block,sr,genre,song_name,counter):
  matplotlib.use('Agg')
  S = librosa.feature.melspectrogram(y=block, sr=sr, n_mels=128,fmax=8000)
  plt.figure(figsize=(6, 3))
  librosa.display.specshow(librosa.power_to_db(S,ref=np.max),y_axis='mel', fmax=8000,x_axis='time')
  # plt.plot()
  song_name=song_name.replace('.mp3','')
  file_name=f'/content/spectrograms6sec/{genre}/'+song_name+str(counter)+'.png'
  plt.savefig(file_name)


def preprocess(file_name,song_name,genre):
  # First load the file
  audio, sr = librosa.load(file_name)
  filename = file_name.replace(".mp3",".wav")

  # Get number of samples for 6 seconds; replace 2 by any number
  buffer = 6 * sr

  samples_total = len(audio)
  samples_wrote = 0
  counter = 1

  while samples_wrote < samples_total:
      
      #check if the buffer is not exceeding total samples 
      if buffer > (samples_total - samples_wrote):
          buffer = samples_total - samples_wrote

      block = audio[samples_wrote : (samples_wrote + buffer)]
      out_filename = "split_" + str(counter) + "_" + filename

      # Write 2 second segment
      save_spectrogram(block,sr,genre,song_name,counter)
      # sf.write('1.wav', block, sr, 'PCM_24')
      counter += 1
      samples_wrote += buffer
      
  
    for index,row in song_df.iterrows():
  preprocess(row["file_path"],row["song"],row["genre"])
  print(f"{row['song']} done")

```

In [ ]:
#IMPORTS put all here
import random

#tf imports for reading file
import numpy as np
import os
import PIL

import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

#extras
print(tf.__version__)

*italicized text*#KNN

In [ ]:
#A5 stuff dont touch
# from google.colab import drive
# from alive_progress import alive_bar

# drive.mount('/content/drive',force_remount=True)

spec_path = 'content/spectrograms6sec'

genre_list=os.listdir(spec_path)
genre_list




In [ ]:
%rm -rf content/spectrograms6secnew

In [ ]:
os.mkdir('content/spectrograms6secnew')
dest_path='content/spectrograms6secnew'

for genre in genre_list:
  os.mkdir(f'content/spectrograms6secnew/{genre}')

In [ ]:
import shutil

#no of samples per class
n=3000
cnt=0

random.seed(123)

for genre in genre_list:
  genre_path=os.path.join(spec_path,genre)
  dest_genre_path=os.path.join(dest_path,genre)

  #shuffling and selecting 3000 samples
  genre_songs=os.listdir(genre_path)
  random.shuffle(genre_songs)
  selected_spec=genre_songs[:n]
  
  for song in selected_spec:
      source=os.path.join(genre_path,song)
      dest=os.path.join(dest_genre_path,song)
      d= shutil.copyfile(source, dest)




In [ ]:
import pathlib

data_dir=pathlib.Path(dest_path)
data_dir

In [ ]:
#adding dataset
batch_size=64
image_height=75
image_width=150


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size)


In [ ]:
class_names = train_ds.class_names
print(class_names)

import matplotlib.pyplot as plt

plt.figure(figsize=(45, 15))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

In [ ]:
model = Sequential()

In [ ]:
num_classes = 10

# 16 - number of filters
# (3,3) - filter size
# 1 - stride of filter
# relu activation - converts negative numbers to 0 (rectified linear activation)
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(75,150,3)))
BatchNormalization()
# condenses data 
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu' ))
BatchNormalization()
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu' ))
BatchNormalization()
model.add(MaxPooling2D())

# condensing into single output
model.add(Flatten())

model.add(Dense(256, activation='relu'))
BatchNormalization()
model.add(Dropout(rate=0.2))
# softmax activation - maps to a multiclass classification
model.add(Dense(1, activation='softmax'))

In [ ]:
model.compile('adam',loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])
model.summary()

In [ ]:
logdir='logs'
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

In [ ]:
hist=model.fit(train_ds,epochs=100,validation_data=val_ds,callbacks=[tensorboard_callback,early])

# KNN

In [ ]:
#ayushmaan stuff

# SVM

In [ ]:
#ayush stuff